In [1]:
import os
import json
import re
from tqdm import tqdm
import pickle
import itertools
import networkx as nx
from matplotlib import pyplot as plt
import networkx

In [3]:
with open('extracted_words_deeppavlov.pkl', 'rb') as f:
    extracted_words_deeppavlov = pickle.load(f)

In [4]:
extracted_words_deeppavlov

{'0000000002': ['ирбис', 'бумаги', 'воскресенье'],
 '0000000003': ['whisper systems', 'whats app'],
 '0000000004': ['samsung'],
 '0000000005': [],
 '0000000006': [],
 '0000000007': [],
 '0000000010': [],
 '0000000012': [],
 '0000000013': [],
 '0000000014': [],
 '0000000015': ['tj', 'apple'],
 '0000000016': [],
 '0000000017': ['whatsapp'],
 '0000000018': [],
 '0000000019': [],
 '0000000020': ['tj', 'apple', 'ооо « тджорнал', 'tjournal'],
 '0000000021': ['лдпр'],
 '0000000022': ['газпром - медиа', 'эха москвы', 'эха'],
 '0000000023': [],
 '0000000024': [],
 '0000000025': [],
 '0000000029': ['европейского альянса за права человека в северной корее "'],
 '0000000030': ['guvera'],
 '0000000031': ['google'],
 '0000000032': [],
 '0000000033': ['цб'],
 '0000000034': [],
 '0000000035': ['the village'],
 '0000000037': [],
 '0000000038': ['apple'],
 '0000000039': ['uber'],
 '0000000040': [],
 '0000000041': [],
 '0000000042': [],
 '0000000043': ['wi - fi'],
 '0000000044': ['huawei'],
 '0000000045'

In [2]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, EdgesAndLinkedNodes, NodesAndLinkedEdges, LabelSet
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from networkx.algorithms import community

In [9]:
path = './final_proj/'
files = sorted(os.listdir(path))

In [10]:
N = len(files)

In [11]:
N

298694

In [12]:
with open('companies_vc.txt', 'r') as f:
    companies_vc = f.read()

In [13]:
len(companies_vc.split('\n'))

476

In [14]:
companies_vc = companies_vc.split('\n')

In [15]:
with open('companies_habr.txt', 'r') as f:
    companies_habr = f.read()

In [16]:
len(companies_habr.split('\n'))

25244

In [17]:
companies_habr = companies_habr.split('\n')

In [18]:
companies = set(companies_vc).union(set(companies_habr))

In [19]:
data = 0
with open(path + files[100100]) as json_file:
    data = json.load(json_file)

In [20]:
data.keys()

dict_keys(['message', 'result'])

In [21]:
data['result'].keys()

dict_keys(['id', 'url', 'author', 'badges', 'commentsCount', 'commentsSeenCount', 'favoritesCount', 'cover', 'date', 'dateRFC', 'date_favorite', 'last_modification_date', 'hitsCount', 'intro', 'introInFeed', 'isEnabledComments', 'isEnabledLikes', 'isFavorited', 'isRepost', 'likes', 'subsite', 'similar', 'title', 'type', 'entryContent', 'commentatorsAvatars', 'webviewUrl', 'isPinned', 'canEdit', 'etcControls', 'highlight', 'blocks', 'subscribedToTreads', 'is_show_thanks', 'is_still_updating', 'is_filled_by_editors', 'isEditorial', 'audioUrl', 'hotness', 'commentEditor', 'summarize'])

In [22]:
data['result']['likes']

{'is_liked': 0, 'count': 21, 'summ': 21, 'is_hidden': False}

In [23]:
data['result']['intro']

'Пользователи TJ и Nintendo Switch делятся мнением о октябрьских релизах.'

In [24]:
data['result']['blocks']

[{'type': 'text',
  'data': {'text': 'Пользователи TJ и Nintendo Switch делятся мнением о октябрьских релизах.',
   'text_truncated': '<<<same>>>'},
  'cover': True,
  'hidden': False,
  'anchor': ''},
 {'type': 'media',
  'data': {'items': [{'title': '\u200bАрт к игре «The Witcher 3: Wild Hunt»',
     'author': '',
     'image': {'type': 'image',
      'data': {'uuid': '7d93dd4e-8f42-f9a5-b0b6-c9d1a9ae5329',
       'width': 1920,
       'height': 1016,
       'size': 469401,
       'type': 'jpg',
       'color': '344543',
       'hash': '',
       'external_service': []}}}],
   'with_background': False,
   'with_border': False},
  'cover': True,
  'hidden': False,
  'anchor': ''},
 {'type': 'text',
  'data': {'text': 'Вот и подошла к концу середина осени. Октябрь был не самым насыщенным месяцем для гибридной консоли, однако, несколько громких и необычных релизов всё-таки было. Пользователи TJ поделились своими впечатлениями для ежемесячной рубрики.',
   'text_truncated': '<<<same>>>'}

In [25]:
extracted_words = {}
for j in tqdm(range(N)):
    with open(path + files[j]) as json_file:
        data = json.load(json_file)
    num_blocks = len(data['result']['blocks'])
    extracted_words_set = set()
    
    for i in range(num_blocks):
        if data['result']['blocks'][i]['type'] == 'text' or data['result']['blocks'][i]['type'] == 'header':
            extracted_words_set = extracted_words_set.union(set(re.findall(r'«[\w]+»', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'"[\w]+"', data['result']['blocks'][i]['data']['text'])))
            #print(re.findall(r'[\s]?[A-Z][A-Za-z]+[\s]|[\s]?[А-Я][А-Яа-я]+[\s]', data['result']['blocks'][i]['data']['text']))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][A-Za-z]*[\.][A-Z][A-Za-z]*|[А-Я][А-Яа-я]*[\.][А-Я][А-Яа-я]*', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z]+[^a-z]|[А-Я]+[^а-я]', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][a-z]+|[А-Я][а-я]+', data['result']['blocks'][i]['data']['text'])))
            
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][a-z]+[\s-][A-Z][a-z]+|[А-Я][а-я]+[\s-][А-Я][а-я]+', data['result']['blocks'][i]['data']['text'])))
            extracted_words_set = extracted_words_set.union(set(re.findall(r'[A-Z][a-z]+\s&\s[A-Z][a-z]+|[A-Z][a-z]+&[A-Z][a-z]+', data['result']['blocks'][i]['data']['text'])))

    extracted_words[files[j]] = [item.lower() for item in extracted_words_set]
    #        print(data['result']['blocks'][i]['data']['text'].split())
        

100%|██████████████████████████████████| 298694/298694 [12:11<00:00, 408.29it/s]


In [31]:
re.findall(r'[A-Z][a-z]+[\s-][A-Z][a-z]+|[А-Я][а-я]+[\s-][А-Я][а-я]+', '.Whats App')

['Whats App']

In [36]:
'в '.strip('«.» ,')

'в'

In [27]:
extracted_words

{'0000000002': ['«воскресенье»',
  'казачьим',
  'f0',
  'f)',
  'в ',
  'корреспондент',
  'воскресенье',
  'fwww',
  'dhttp',
  'f1',
  'fbeta',
  'f%',
  'fredirect',
  '«ирбис»',
  'ирбис»',
  'бумаги',
  'f2',
  'петербурге',
  'a%',
  'furl',
  '«бумаги»'],
 '0000000003': ['f)',
  'сноуденом',
  'furl',
  'fwhatsapp',
  'fblog',
  'dhttps',
  'fgithub',
  'эдвардом',
  'fbeta',
  'f%',
  'text',
  'fredirect',
  'app',
  'эдвардом сноуденом',
  'используемый',
  'whisper systems',
  'android',
  'последняя',
  'whats',
  'fwhispersystems',
  'пока',
  'whisper',
  'secure',
  'systems',
  'a%',
  'ftextsecure',
  'whats app'],
 '0000000004': ['plus',
  'фернандес',
  'подросток',
  'бразилии',
  'samsung',
  'фернандес фильо',
  'фильо',
  'гиннесса',
  'phone',
  'предыдущий',
  'fleksy',
  'марселем',
  'samsung galaxy',
  'galaxy',
  'подросток марсель',
  's4',
  'марсель'],
 '0000000005': ['чо'],
 '0000000006': ['ваши', 'лихачев'],
 '0000000007': [],
 '0000000010': ['tj ', '

In [22]:
#with open('extracted_words.pkl', 'wb') as f:
#    pickle.dump(extracted_words, f)

In [213]:
'fl' in companies_vc

False

In [214]:
stop_words = set(['set', 
                  'далее', 
                  'развитие', 
                  'алгоритмы', 
                  'мария', 
                  'red', 
                  'main', 
                  'стартап', 
                  'бюро', 
                  'id', 
                  'поиск', 
                  'ясно',
                  'интерес',
                  'we',
                  'fl',
                  'too',
                  'успех',
                  'система'])

In [236]:
companies = set(companies_vc)

In [237]:
companies.add('mail.ru')
companies.add('mail')
companies.add('mail.ru group')
companies.add('мэйл')
#companies.add('facebook')
#companies.add('meta')
companies = companies - stop_words

In [238]:
extracted_companies = {}
for key in extracted_words.keys():
    for word in extracted_words[key]:
        if word in companies:
            if not key in extracted_companies.keys():
                extracted_companies[key] = []
            extracted_companies[key].append(word)

In [239]:
extracted_companies

{'0000000013': ['tj'],
 '0000000020': ['tj'],
 '0000000097': ['яндекс'],
 '0000000114': ['mail'],
 '0000000116': ['tj'],
 '0000000121': ['intel'],
 '0000000153': ['mail', 'mail.ru'],
 '0000000166': ['yota'],
 '0000000220': ['вконтакте'],
 '0000000228': ['coub'],
 '0000000263': ['mail'],
 '0000000301': ['яндекс'],
 '0000000330': ['tj'],
 '0000000342': ['mail', 'qiwi'],
 '0000000349': ['microsoft'],
 '0000000350': ['tj'],
 '0000000375': ['tj'],
 '0000000414': ['lamoda'],
 '0000000452': ['автоваз'],
 '0000000466': ['intel'],
 '0000000467': ['яндекс', 'mail'],
 '0000000472': ['яндекс'],
 '0000000483': ['metro'],
 '0000000510': ['microsoft'],
 '0000000532': ['tj'],
 '0000000554': ['yota'],
 '0000000590': ['yota'],
 '0000000593': ['tj'],
 '0000000596': ['tj'],
 '0000000644': ['microsoft'],
 '0000000652': ['tj'],
 '0000000665': ['одноклассники'],
 '0000000719': ['mail'],
 '0000000759': ['ростелеком'],
 '0000000806': ['тинькофф'],
 '0000000882': ['yota'],
 '0000000921': ['одноклассники'],
 '00

In [240]:
#with open('extracted_companies.pkl', 'wb') as f:
#    pickle.dump(extracted_companies, f)

In [241]:
edges = []
for key in extracted_companies.keys():
    if len(extracted_companies[key]) > 1:
        for element in itertools.product(*[extracted_companies[key], extracted_companies[key]]):
            edges.append(element)

In [242]:
edges

[('mail', 'mail'),
 ('mail', 'mail.ru'),
 ('mail.ru', 'mail'),
 ('mail.ru', 'mail.ru'),
 ('mail', 'mail'),
 ('mail', 'qiwi'),
 ('qiwi', 'mail'),
 ('qiwi', 'qiwi'),
 ('яндекс', 'яндекс'),
 ('яндекс', 'mail'),
 ('mail', 'яндекс'),
 ('mail', 'mail'),
 ('intel', 'intel'),
 ('intel', 'microsoft'),
 ('microsoft', 'intel'),
 ('microsoft', 'microsoft'),
 ('mail', 'mail'),
 ('mail', 'mail.ru'),
 ('mail.ru', 'mail'),
 ('mail.ru', 'mail.ru'),
 ('одноклассники', 'одноклассники'),
 ('одноклассники', 'ivi'),
 ('ivi', 'одноклассники'),
 ('ivi', 'ivi'),
 ('microsoft', 'microsoft'),
 ('microsoft', 'манифест'),
 ('манифест', 'microsoft'),
 ('манифест', 'манифест'),
 ('microsoft', 'microsoft'),
 ('microsoft', 'intel'),
 ('intel', 'microsoft'),
 ('intel', 'intel'),
 ('одноклассники', 'одноклассники'),
 ('одноклассники', 'вконтакте'),
 ('вконтакте', 'одноклассники'),
 ('вконтакте', 'вконтакте'),
 ('вконтакте', 'вконтакте'),
 ('вконтакте', 'одноклассники'),
 ('одноклассники', 'вконтакте'),
 ('одноклассники'

In [258]:
edges_ = {}
for item in tqdm(edges):
    if item[0] != item[1]:
        if (not (item[0], item[1]) in edges_.keys()) and (not (item[1], item[0]) in edges_.keys()):
            edges_[item] = 0.0
        else:
            if (item[0], item[1]) in edges_.keys():
                edges_[item] += 1.0
            elif (item[1], item[0]) in edges_.keys():
                edges_[(item[1], item[0])] += 1.0

100%|█████████████████████████████████| 12437/12437 [00:00<00:00, 375443.96it/s]


In [259]:
edges_

{('mail', 'mail.ru'): 735.0,
 ('mail', 'qiwi'): 7.0,
 ('яндекс', 'mail'): 497.0,
 ('intel', 'microsoft'): 139.0,
 ('одноклассники', 'ivi'): 15.0,
 ('microsoft', 'манифест'): 1.0,
 ('одноклассники', 'вконтакте'): 51.0,
 ('яндекс', 'яндекс.маркет'): 63.0,
 ('медиалогия', 'вконтакте'): 1.0,
 ('ozon', 'mail.ru'): 7.0,
 ('ozon', 'microsoft'): 11.0,
 ('ozon', 'mail'): 23.0,
 ('ozon', 'яндекс'): 61.0,
 ('mail.ru', 'microsoft'): 27.0,
 ('mail.ru', 'яндекс'): 211.0,
 ('microsoft', 'mail'): 65.0,
 ('microsoft', 'яндекс'): 91.0,
 ('microsoft', 'tj'): 13.0,
 ('yota', 'intel'): 3.0,
 ('мегафон', 'yota'): 25.0,
 ('яндекс', 'gett'): 67.0,
 ('мтс', 'мегафон'): 3.0,
 ('лайфхакер', 'spotify'): 7.0,
 ('яндекс', 'tj'): 101.0,
 ('ebay', 'microsoft'): 5.0,
 ('mail', 'одноклассники'): 207.0,
 ('яндекс', 'одноклассники'): 149.0,
 ('яндекс', 'вконтакте'): 73.0,
 ('яндекс', 'контур'): 11.0,
 ('tj', 'пик'): 1.0,
 ('яндекс', 'spotify'): 283.0,
 ('вконтакте', 'tj'): 13.0,
 ('okko', 'ivi'): 45.0,
 ('microsoft', 'sp

In [260]:
G = nx.Graph()
for key in edges_.keys():
    G.add_weighted_edges_from([(key[0], key[1], edges_[key])])

In [261]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

In [262]:
number_to_adjust_by = 3
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in networkx.degree(G)])
networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

In [267]:
from networkx.algorithms import community
communities = community.greedy_modularity_communities(G, weight='weight', resolution=0.8)

In [268]:
# Create empty dictionaries
modularity_class = {}
modularity_color = {}
colors = (
'#3288bd',
'#66c2a5',
'#abdda4',
'#e6f598',
'#fee08b',
'#fdae61',
'#f46d43',
'#d53e4f',   
'#8c2d04',
)
#Loop through each community in the network
for community_number, community in enumerate(communities[:8]):
    #For each member of the community, add their community number and a distinct color
    for name in community: 
        modularity_class[name] = community_number
        modularity_color[name] = colors[community_number]

In [269]:
networkx.set_node_attributes(G, modularity_class, 'modularity_class')
networkx.set_node_attributes(G, modularity_color, 'modularity_color')

In [271]:
#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'
edge_color = 'gray'

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'modularity_color'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Blues8

#Choose a title!
title = 'Companies graph'


#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Companies", "@index"),
        ("Degree", "@degree"),
         ("Modularity Class", "@modularity_class"),
      #  ("Modularity Color", "$color[swatch]:modularity_color"),
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as category from attribute)
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=color_by_this_attribute)



#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.3, line_color=edge_color, line_width=1)
#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(network_graph)

#Add Labels
x, y = zip(*network_graph.layout_provider.graph_layout.values())
node_labels = list(G.nodes())
source = ColumnDataSource({'x': x, 'y': y, 'name': [node_labels[i] for i in range(len(x))]})
labels = LabelSet(x='x', y='y', text='name', source=source, background_fill_color='white', text_font_size='10px', background_fill_alpha=.7)
plot.renderers.append(labels)

show(plot)
#save(plot, filename=f"{title}.html")

In [273]:
with open('edges_vc.pkl', 'wb') as f:
    pickle.dump(edges, f)

In [274]:
with open('edges_vc_.pkl', 'wb') as f:
    pickle.dump(edges_, f)

In [272]:
len(G.nodes)

116

In [275]:
len(G.edges)

631